In [1]:
import pandas as pd
import time
import io
from api_request_funtions import ApiGet
from bq_transfers import BqDataTransfers 
from main import fetch_and_save
import json

/Users/shami/personal_projects/vexis_authLive/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
thrivecart_get = ApiGet(http='thrivecart.com', api_key='TZ5TJYBR-FDB85IBI-0RFTB00N-VQ7ZFY2S')
thrivecart_save = BqDataTransfers(gcp_project_id= 'arboreal-cat-451816-n0', bq_data_set = 'thrive_cart')
bq_client = thrivecart_save.get_bq_client("/Users/shami/Library/Mobile Documents/com~apple~CloudDocs/Personal Projects/vexis/vexis_bq_writter.json")

# split products

In [ ]:
# def fetch_transactions(event, context):
    # Fetch the total count of transactions from the API
x = thrivecart_get.fetch_data(end_point=f'/api/external/transactions?page=0&perPage=0&query=&transactionType=any')
total_records_api = x['meta']['total']
total_records_bq = bq_client.get_table('thrive_cart.transactions').num_rows

# Calculate the difference and determine the number of pages to fetch
records_to_fetch = max(0, total_records_api - (total_records_bq))
pages = list(range(1, (records_to_fetch // 100) + 2)) if records_to_fetch > 0 else []
print(len(pages))
pages = reversed(pages)
print(pages)
# Fetch new transactions

BIGQUERY_SCHEMA = [
    {"name": "quantity", "type": "INTEGER"},
    {"name": "item_type", "type": "STRING"},
    {"name": "processor", "type": "STRING"},
    {"name": "currency", "type": "STRING"},
    {"name": "coupon", "type": "STRING"},
    {"name": "invoice_id", "type": "INTEGER"},
    {"name": "order_id", "type": "INTEGER"},
    {"name": "internal_subscription_id", "type": "INTEGER"},
    {"name": "item_pricing_option_id", "type": "INTEGER"},
    {"name": "time", "type": "TIMESTAMP"},
    {"name": "item_id", "type": "INTEGER"},
    {
        "name": "customer",
        "type": "RECORD",
        "fields": [
            {"name": "email", "type": "STRING"},
            {"name": "name", "type": "STRING"}
        ]
    },
    {"name": "amount", "type": "INTEGER"},
    {"name": "transaction_info", "type": "STRING"},
    {"name": "subscription_id", "type": "STRING"},
    {"name": "item_name", "type": "STRING"},
    {"name": "transaction_type", "type": "STRING"},
    {"name": "reference", "type": "STRING"},
    {"name": "item_pricing_option_name", "type": "STRING"},
    {"name": "date", "type": "DATE"},
    {"name": "campaign_id", "type": "STRING"},
    {"name": "transaction_id", "type": "STRING"},
    {"name": "event_id", "type": "INTEGER"},
    {"name": "base_product", "type": "INTEGER"},
    {"name": "timestamp", "type": "INTEGER"}
]

for page in pages:
    x = thrivecart_get.fetch_data(
        end_point=f'/api/external/transactions?page={page}&perPage=100&query=&transactionType=any'
    )
    transactions = x.get("transactions", [])

    if transactions:
        # Convert to NDJSON format
        ndjson_lines = "\n".join(json.dumps(transaction) for transaction in transactions)
        ndjson_buffer = io.StringIO(ndjson_lines)
        try:
            # Upload to BigQuery, ensuring no duplicates based on transaction_id
            thrivecart_save.start_transfer_json(
                bq_client=bq_client,
                file=ndjson_buffer,
                destination_table='transactions',
                write_options='append',
                schema=BIGQUERY_SCHEMA
            )
        except:
               thrivecart_save.start_transfer_json(
                bq_client=bq_client,
                file=ndjson_buffer,
                destination_table='transactions_errors',
                write_options='append'
            )


    time.sleep(0.5)


516
